## Chap 2.6: Feature engineering


The process of feature engineering is of manipulating, transforming, selecting raw data into features that can be used in statistical analysis of prediction.

* statistical features
* temporal features
* spectral features (Fourier and Wavelet transforms)

We will download seismic data and work using the TSFEL package to extract features. Because TSFEL is not in the conda registry, we have to install it using pip.

In [ ]:
%pip install tsfel

In [ ]:
# Import modules for seismic data and feature extraction
import obspy
import numpy as np
import tsfel
import matplotlib.pyplot as plt
import pandas as pd

import obspy.clients.fdsn.client as fdsn
from obspy import UTCDateTime



We will use a time series from a seismic record of the 10/7/2022 M4.4 Earthquake in Oregon. The earthquake had a moderate size but was felt by many citizens.

<div>
<img src="uw61886506_ciim_geo.jpeg" width="500"/>
</div>

We will first download data from the PNSN seismic station UW.MRIN.

In [ ]:
# Solution
network = 'UW'
station = 'RATT'
channel = 'HHZ'# this channel gives a low frequency, 1Hz signal.
Tstart = UTCDateTime(2021,7,29,6,15)
Tend = Tstart+7200# UTCDateTime(year=2022, month=10, day=8)
fdsn_client = fdsn.Client('IRIS')
Z = fdsn_client.get_waveforms(network=network, station=station, location='--', channel=channel, starttime=Tstart, \
    endtime=Tend, attach_response=True)
Z.detrend()
Z.merge()

In [ ]:
# plot the time series
T = np.linspace(0,(Tend-Tstart),Z[0].stats.npts)
%matplotlib inline
plt.plot(T,Z[0].data)
plt.grid(True)
plt.xlabel('Time since origin (s)')
plt.title('M8.2 2021/07/29 recorded at UW.RATT.HHZ')
plt.ylabel('Counts')
plt.show()


Now, only seismologists work on obspy stream, so we will convert the data into numpy arrays:

In [ ]:
z = np.asarray(Z[0].data)

## 2.6.2 Statistical Features


### The mean
<div>
<img src="mean.png" alt="mean" height=300  />
</div>
Image taken from this [blog](https://gregorygundersen.com/blog/2020/04/11/moments/).

The mean is the sum of the values divided by the number of data points. It is the first raw moment of a distribution.

### The Variance
 <div>
<img src="variance.png" alt="variance" height=200  />
</div>

The variance is the second centralized moment.

$\sigma = \sum_{i=1}^n (t_i - \mu)^2 F_i / N$



In [ ]:
cfg_file = tsfel.get_features_by_domain('spectral')         

In [ ]:
print(cfg_file)

In [ ]:
z_df=pd.DataFrame({'data':z})

In [ ]:
%timeit X = tsfel.time_series_features_extractor(cfg_file, z_df,fs=100,single_window=True)

We will save the names of the features in a list ``Lfeature``

In [ ]:
Lfeature = X.keys()

Now let's plot the features

In [ ]:
X[Lfeature[1]]